In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
data_unida=pd.concat([data,data1])
data_unida.shape

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_13992/918402829.py:1: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_13992/918402829.py:2: DtypeWarning: Columns (5,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Raw/dataset_para_modelas_parte1.csv')


(634682, 49)

# **INFORMACIÓN GENERAL DE LOS DATOS**
---

In [3]:
data_unida.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634682 entries, 0 to 317340
Data columns (total 49 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             634682 non-null  int64  
 1   cod_persona            634682 non-null  int64  
 2   mes                    634682 non-null  object 
 3   pais                   634570 non-null  object 
 4   sexo                   634566 non-null  object 
 5   edad                   634682 non-null  object 
 6   fecha1                 634570 non-null  object 
 7   xti_empleado           634570 non-null  object 
 8   xti_nuevo_cliente      634570 non-null  float64
 9   num_antiguedad         634682 non-null  object 
 10  xti_rel                634570 non-null  float64
 11  fec_ult_cli_1t         40 non-null      object 
 12  xti_rel_1mes           629758 non-null  float64
 13  tip_rel_1mes           629758 non-null  object 
 14  indresi                634570 non-nu

# **CANTIDAD DE NULOS**
---

In [6]:
data_unida.isna().sum() #/ len(data_unida)

Unnamed: 0                    0
cod_persona                   0
mes                           0
pais                        112
sexo                        116
edad                          0
fecha1                      112
xti_empleado                112
xti_nuevo_cliente           112
num_antiguedad                0
xti_rel                     112
fec_ult_cli_1t           634642
xti_rel_1mes               4924
tip_rel_1mes               4924
indresi                     112
indext                      112
des_canal                  5892
xti_extra                   112
tip_dom                     112
cod_provincia              3278
xti_actividad_cliente       112
imp_renta                105170
id_segmento                6028
mean_engagement            4924
ind_prod1                     0
ind_prod2                     0
ind_prod3                     0
ind_prod4                     0
ind_prod5                     0
ind_prod6                     0
ind_prod7                     0
ind_prod

# **ANALISIS DESCRIPTIVO DE VARIABLES CATEGÓRICAS**
---

In [7]:
data_unida.describe(include='O')

,mes,pais,sexo,edad,fecha1,xti_empleado,num_antiguedad,fec_ult_cli_1t,tip_rel_1mes,indresi,indext,des_canal,xti_extra,id_segmento
count,634682,634570,634566,634682,634570,634570,634682,40,629758,634570,634570,628790,634570,628654
unique,10,47,2,213,5585,4,496,8,4,2,2,126,2,3
top,2015-09-28,ES,V,23,2014-10-03,N,21,2015-07-30,I,S,N,KHE,N,02 - PARTICULARES
freq,86340,631404,348198,23934,3070,634054,11528,14,322920,631404,605098,209394,633106,372398


# **ANALISIS DESCRIPTIVO DE VARIABLES NUMÉRICAS**
---

In [12]:
#Primera mitad del dataset
data_unida.iloc[:,:round(data_unida.shape[1]/2)].describe()

,Unnamed: 0,cod_persona,xti_nuevo_cliente,xti_rel,xti_rel_1mes,tip_dom,cod_provincia,xti_actividad_cliente,imp_renta,mean_engagement
count,634682.000000,6.346820e+05,634570.000000,634570.000000,629758.000000,634570.0,631404.000000,634570.000000,5.295120e+05,629758.000000
mean,161494.522060,7.535270e+05,0.028246,1.006177,1.000292,1.0,26.608868,0.494559,1.369546e+05,6.439201
std,93016.280074,4.121444e+05,0.165675,0.778042,0.025074,0.0,12.734544,0.499971,3.339431e+05,1.324006
min,0.000000,8.795000e+04,0.000000,1.000000,0.000000,1.0,1.000000,0.000000,4.559550e+03,0.218520
25%,80989.000000,3.989580e+05,0.000000,1.000000,1.000000,1.0,15.000000,0.000000,6.821199e+04,5.117500
50%,161691.000000,6.422680e+05,0.000000,1.000000,1.000000,1.0,28.000000,0.000000,1.014297e+05,6.463518
75%,242079.000000,1.120981e+06,0.000000,1.000000,1.000000,1.0,34.000000,1.000000,1.568756e+05,7.359949
max,322352.000000,1.537800e+06,1.000000,99.000000,4.000000,1.0,52.000000,1.000000,2.889440e+07,31.967536


In [13]:
#Segunda mitad del dataset
data_unida.iloc[:,round(data_unida.shape[1]/2):].describe()

,ind_prod1,ind_prod2,ind_prod3,ind_prod4,ind_prod5,ind_prod6,ind_prod7,ind_prod8,ind_prod9,ind_prod10,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
count,634682.000000,634682.0,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,...,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,634682.000000,634620.000000,634620.000000,634682.00000,634682.000000
mean,0.000113,0.0,0.714071,0.000551,0.088145,0.011291,0.010462,0.149801,0.049751,0.002950,...,0.010440,0.003107,0.057827,0.051585,0.029133,0.004289,0.061167,0.066244,0.13955,0.073955
std,0.010650,0.0,0.451856,0.023477,0.283506,0.105656,0.101747,0.356877,0.217430,0.054229,...,0.101641,0.055654,0.233417,0.221188,0.168179,0.065348,0.239637,0.248709,0.34652,0.261698
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
